# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
 # Data Science
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys_prof import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
df = pd.read_csv("output_data/cities_booooooth.csv")

# Display sample data
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,George Town,5.4112,100.3354,78.73,93,20,2.30,1007,MY,1732577126
1,1,Isafjordur,66.0755,-23.1240,26.92,83,9,10.40,1004,IS,1732577077
2,2,Port-aux-Français,-49.3500,70.2167,37.80,71,67,37.18,987,TF,1732577058
3,3,Wailua Homesteads,22.0669,-159.3780,81.12,70,0,14.97,1015,US,1732577355
4,4,Badger,64.8000,-147.5333,21.45,84,100,0.00,1010,US,1732577356


In [6]:
# Date Cleaning w/dates
df["Date"] = pd.to_datetime(df.Date * 1e9)
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,George Town,5.4112,100.3354,78.73,93,20,2.30,1007,MY,2024-11-25 23:25:26
1,1,Isafjordur,66.0755,-23.1240,26.92,83,9,10.40,1004,IS,2024-11-25 23:24:37
2,2,Port-aux-Français,-49.3500,70.2167,37.80,71,67,37.18,987,TF,2024-11-25 23:24:18
3,3,Wailua Homesteads,22.0669,-159.3780,81.12,70,0,14.97,1015,US,2024-11-25 23:29:15
4,4,Badger,64.8000,-147.5333,21.45,84,100,0.00,1010,US,2024-11-25 23:29:16


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City_ID     58 non-null     int64         
 1   City        58 non-null     object        
 2   Lat         58 non-null     float64       
 3   Lng         58 non-null     float64       
 4   Max Temp    58 non-null     float64       
 5   Humidity    58 non-null     int64         
 6   Cloudiness  58 non-null     int64         
 7   Wind Speed  58 non-null     float64       
 8   Pressure    58 non-null     int64         
 9   Country     58 non-null     object        
 10  Date        58 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(4), object(2)
memory usage: 5.1+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 70
max_temp = 80
max_wind = 10

# Drop any rows with null values
df2 = df.dropna()

mask = (df2["Max Temp"] >= min_temp) & (df2["Max Temp"] < max_temp) & (df2["Wind Speed"] < max_wind)
df2 = df2.loc[mask].reset_index(drop=True)

# Display sample data
df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,George Town,5.4112,100.3354,78.73,93,20,2.30,1007,MY,2024-11-25 23:25:26
1,25,Tabou,4.4230,-7.3528,78.24,85,7,5.86,1014,CI,2024-11-25 23:29:41
2,46,Parakou,9.3372,2.6303,72.18,22,27,5.68,1013,BJ,2024-11-25 23:30:05
3,50,La Foa,-21.7108,165.8276,78.06,78,85,4.81,1012,NC,2024-11-25 23:30:09
4,53,Monrovia,6.3005,-10.7969,78.58,85,32,6.53,1014,LR,2024-11-25 23:30:13


In [12]:
hotel_rows = []

for index, row in df2.iterrows():
    # from the Weather CSV
    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Set the parameters for the type of place
    categories = "accommodation.hotel"
    radius = 10000 # 10km
    
    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5
    
    # set up a parameters dictionary
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key    
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Run request
    try:
        response = requests.get(base_url, params=params)
        # print(response.status_code)
        data = response.json()
        
        # Print the results
        results = data.get("features", [])
        
        # Resiliency/Error Handling
        if len(results) > 0:
            place = results[0]
    
            # normalize data
            address = place.get("properties", {}).get("formatted")
            name = place.get("properties", {}).get("name")
            distance = place.get("properties", {}).get("distance")
            elev = place.get("properties", {}).get("ele")
            website = place.get("properties", {}).get("website")
        
            # return object
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": address,
                "name": name,
                "distance": distance,
                "elevation": elev,
                "website": website
            }
        else:
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }
    except Exception as e:
        print(e)
        hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }

    # append to hotel list
    hotel_rows.append(hotel_row)

In [14]:
hotel_df = pd.DataFrame(hotel_rows)
hotel_df

,city,country,latitude,longitude,address,name,distance,elevation,website
0,George Town,MY,5.4112,100.3354,"Page 63 hostel, Beach Street, 10300 George Tow...",Page 63 hostel,139,None,None
1,Tabou,CI,4.4230,-7.3528,"hôtel le rochet, A7, Tabou, Côte d'Ivoire",hôtel le rochet,722,None,None
2,Parakou,BJ,9.3372,2.6303,"Hôtel Iya-Oasis, RNIE 6, Parakou, Benin",Hôtel Iya-Oasis,1226,None,None
3,La Foa,NC,-21.7108,165.8276,"Rue Pierre Bergas, 98880 La Foa, France",None,236,None,None
4,Monrovia,LR,6.3005,-10.7969,"Bella Casa Hotel, Tubman Boulevard, Monrovia, ...",Bella Casa Hotel,811,None,None


In [17]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "city",
    hover_cols=["city", "country", "name", "address"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,country,name,address)

### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE